In [1]:
import subprocess
import re
import json

# def calculate_score(error_count: int) -> float:|
#     if error_count == 0:
#         return 5.0
#     elif error_count == 1:
#         return 4.5
#     elif error_count in [2, 3]:
#         return 4.0
#     elif error_count == 4:
#         return 3.5
#     elif error_count in [5, 6]:
#         return 3.0
#     elif error_count in [7, 8]:
#         return 2.5
#     elif error_count == 9:
#         return 2.0
#     elif error_count in [10, 11]:
#         return 1.5
#     elif error_count in [12, 13]:
#         return 1.0
#     elif error_count in [14, 15]:
#         return 0.5
#     else:
#         return 0.0

def calculate_score(error_count: int) -> float:
    """Calculates score by deducting 0.75 marks for each error."""
    score = max(5.0 - (0.75 * error_count), 0.0)
    return round(score, 2)  # Rounds to 2 decimal places

def check_java_syntax(java_file_path: str) -> tuple[int, float, dict]:
    """
    Reads Java code from a file and checks for syntax errors using `javac`.
    Returns the error count, score, and a dictionary of errors.
    """
    error_dict = {}  # Dictionary to store errors by line number
    
    try:
        # Compile the Java file using javac
        result = subprocess.run(
            ["javac", java_file_path],
            capture_output=True,
            text=True
        )
        
        error_count = 0
        if result.stderr:
            print("Compilation Errors:")
            errors = result.stderr.strip().split('\n')
            
            # Process each error message
            for error in errors:
                # Skip the last line which contains error count
                if 'error' in error and not re.search(r'\d+ errors?$', error):
                    # Extract line number using regex
                    line_match = re.search(r':(\d+):', error)
                    if line_match:
                        line_num = int(line_match.group(1))
                        # Remove the file path and line number from the error message
                        error_msg = re.sub(r'^.*?:\d+:', '', error).strip()
                        
                        # Add error to dictionary
                        if line_num not in error_dict:
                            error_dict[line_num] = []
                        error_dict[line_num].append(error_msg)
                        print(f"- Line {line_num}: {error_msg}")
            
            # Extract the total error count
            last_line = errors[-1]
            if match := re.search(r'(\d+) errors?', last_line):
                error_count = int(match.group(1))
        else:
            print("No syntax errors found!")

        # Calculate score based on error count
        score = calculate_score(len(error_dict))
        print(f"\nCompiler reported errors: {error_count}")
        print(f"Number of lines with errors: {len(error_dict)}")
        print(f"Score: {score}/5")
        print("\nError Dictionary:")
        print(json.dumps(error_dict, indent=2))
        
        return error_count, score, error_dict
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return -1, 0.0, {}

# # Example usage
# java_file_path = "CricketAnalyticsSolution.java"  # Replace with the actual Java file path
# error_count, score, error_dict = check_java_syntax(java_file_path)


In [ ]:
feedbackPromptAIO = '''You are an expert code evaluator, evaluating code submissions for a Java based Object Oriented Programming test at a university level.
You will be provided with the question and a rubric that describes the criteria for evaluation, with a marking scheme. 
The question is a code sample that the examiner provides, containing a template wherein the student is required to write the code as well as comments and instructions from the examiner's end.
Following this you will be provided with the code submission, along with the response from the Java compiler that runs this code.
Note that the code may be formatted liberally, the specific positioning of the code within the methods are not important.
Code may be present either before or after the comments prepared by the instructor.
You are to evaluate the code based only on logical correctness. You are to ignore any syntax errors that the compiler may have thrown. 
Any syntax errors that you encounter can be treated as correct syntax, and you are to infer the student's logical flow and intention from the code.
You are to return your response as a JSON dictionary containing a detailed, nested evaluation of the student's marks for each line in the rubric.
For each line in the rubric, you are to provide the line as the key and your assigned marks as the value.
DO NOT RETURN ANY ADDITIONAL TEXT ASIDE FROM THE JSON DICTIONARY.
Question: {}
Rubric: {}
Code Submission: {}
Compiler Response: {}
'''

feedbackPromptTwoStepPart1 = '''You are an expert code evaluator, evaluating code submissions for a Java based Object Oriented Programming test at a university level.
You will be provided with the code submission and no context whatsoever. You are to infer the student's logical flow and intention from the code.
You are to ignore any syntaxical errors, and are solely to evaluate the code based on logical correctness.
Code Submission: {}
'''

feedbackPromptTwoStepPart2 = '''You are an expert code evaluator, evaluating code submissions for a Java based Object Oriented Programming test at a university level.
You will be provided a question, a marking rubric, and also with the logical flow and intention of a student's code. 
The question is a code sample that the examiner provides, containing a template wherein the student is required to write the code as well as comments and instructions from the examiner's end.
You are to compare the student's logic with what was asked by the question and the rubric.
Question: {}
Rubric: {}
Logical Flow and Intention: {}
'''

In [1]:
appraisal_generator = """You are an expert code evaluator, evaluating code submissions for a Java based Object Oriented Programming test at a university level.
You will be provided with the question, the student submission, and a document containing the appraisal of a previous evaluator.
You wil also be given the rubric the student is being evaluated on.
You are to make your own appraisal based on the student submission and the question and make changes to the original document if necessary.
You must, under all circumstances, keep the nature of the previous document intact. Add your own points whenver necessary.
Note that you must be extremely liberal with your assessment. Only make changes to the original document if you are absolutely sure you have something relevant to the rubric that hasn't been noted before.
Else return the original document as is.
Question: {}
Rubric: {}
Appraised document: {}
Code Submission: {}
"""

In [2]:
question = open('./files/SS_24_25_CBT/CBT_PART_2_QP.java').read()

In [4]:
question

'import java.util.Random;\nimport java.util.Scanner;\n\npublic class CBT_PART_2_QP implements Runnable {\n    \n\tprivate final Player player1;\n    private final Player player2;\n    private final Player player3;\n    private final Random random = new Random();\n    private boolean gameEnded = false;\n    \n    private final Object lock = new Object(); //Central lock - use this for inter-thread coordination\n\n    public CBT_PART_2_QP(String player1Name, String player2Name, String player3Name) {\n        this.player1 = new Player(player1Name, this);\n        this.player2 = new Player(player2Name, this);\n        this.player3 = new Player(player3Name, this);\n    }\n    \n    /******************** DON\'T MODIFY THE CODE OF THIS METHOD *****************\n     * \n     * This method initializes the game by dealing one visible \n     * card to each player at the start. It ensures all players begin with a \n     * known initial score and sets the game\'s starting state. \n     ************

In [5]:
import os.path

import fitz
from fitz import Document, Page, Rect


input_path = "./files/SS_24_25_CBT/CBT_PART-1-2.pdf"
doc: Document = fitz.open(input_path)

pages = doc[3:5]
text = ""

for page in pages:
    page.clean_contents() 
    text += page.get_text()
print(text)


 
 
 
I. Game Description: 
In PART-2, you will implement a classic card game "21" by implementing it as a multithreaded application. 
In this game players aim to reach a score as close as possible to 21 without exceeding it, while competing 
against each other. This game involves one dealer and exactly three players. The dealer does not play but 
manages the game and announces the final result. Each player is represented by a separate thread, and each 
turn and game action is synchronized to prevent conflicts. Here’s how the game works: 
1. Initial Setup: 
 The dealer thread starts the game by dealing two "cards" to each player thread. Each player 
receives one hidden card, known only to that player, and one visible card, which is displayed 
publicly (as a total visible score). 
 Cards are randomly generated integers between 1 and 10, inclusive, with each value equally 
likely. These values represent the points in this simplified version, so there are no suits or special 
cards. 
2.

In [3]:
rubric = r'''
You are required to implement functionalities for a Cricket Analytics application. The provided template code for this application contains multiple classes and methods related to cricket players, their roles, team information, and various data-handling operations. Your tasks involve implementing several methods, each responsible for performing specific actions like reading data from a file, writing data back, updating player statistics, and filtering data.
Tasks and Methods to be Implemented:
1. RunsComparator: compare Method [2 marks] Write code for comparing runs scored by two players in descending order. Return a negative value if the first player has more runs, a positive value if the second player has more runs, or zero if they have the same number of runs.
2. CricketDataHandler: readPlayersFromFile Method [9 marks] Write code for reading player data from the input CSV file and creating a list of Player objects.
● Step 1: Create an empty list to store player details. [1 mark]
● Step 2: Open the specified file for reading data. [1 mark]
● Step 3: Ignore the first line since it contains the column names. [1 mark]
● Step 4: Read each line one by one until reaching the end of the file. [1 mark]
● Step 5: Split the line into different pieces of information. [1 mark]
● Step 6: Create a new player using this information. [1 mark]
● Step 7: Add the new player to the list. [1 mark]
● Step 8: Close the file after reading all data. [1 mark]
● Step 9: Return the complete list of players. [1 mark]
3. CricketDataHandler: writePlayersToFile Method [4 marks] Write code to write the updated list of players back to the output CSV file. The format of the output file should be the same as that of the input file.
● Step 1: Prepare to write data into the specified file. [1 mark]
● Step 2: Write the column names as the first line of the file. [1 mark]
● Step 3: For each player in the list, convert their details to the desired format. [1 mark]
● Step 4: Write each player's information to the file. [1 mark]
4. CricketDataHandler: updatePlayerStats Method [5 marks] Implement the method to update a player's stats (runs and wickets).
● Step 1: Go through each player in the list. [1 mark]
● Step 2: Check if the current player's name matches the given name. [1 mark]
● Step 3: If it matches, update the player's runs with the new value. Updated value will be the sum of the old runs and the argument runs. For example, if a player had 100 runs and the runs argument (to this method) is 50, their new total should be 150 runs. [1 mark]
● Step 4: Similarly, update the player's wickets with the new value. Updated value will be the sum of the old wickets and the argument wickets. For example, if they had 10 wickets and the wickets argument (to this method) is 2, their new total should be 12 wickets[1 mark]
● Step 5: If no player matches the given name, throw an IllegalArgumentException. [1 mark]
5. CricketDataHandler: calculateTeamAverageRuns Method [5 marks] Write code to calculate the average runs scored by players of a specific team.
● Step 1: Filter players belonging to the specified team. [2 marks]
● Step 2: If no players for this team exist, throw an IllegalArgumentException exception. [1 mark]
● Step 3: Calculate the total runs scored by all players from this team. [1 mark]
● Step 4: Compute and return the average runs scored. [1 mark]
6. TeamFilter: filter Method [5 marks] Write code to filter players by their team.
● Step 1: Create an empty list for players matching the criteria. [1 mark]
● Step 2: Go through each player in the players list. [1 mark]
● Step 3: If the player's team matches the given name, add them to the list. [2 marks]
● Step 4: Return the list containing all matching players. [1 mark]
7. AllRounderStatsFilter: filter Method [5 marks] Write code to filter all-rounder players which satisfy the provided criteria (i.e. filter those all-rounders who have runs and wickets greater than or equal to the runs and wickets specified in the criteria respectively).
● Step 1: Create an empty list for players matching the criteria. [1 mark]
● Step 2: Go through each player in the list. [1 mark]
● Step 3: If the player is an all-rounder and meets the given criteria for both runs and wickets, add them to the list. [2 marks]
● Step 4: Return the list containing all matching players. [1 mark]'''

In [7]:
studentFilepath = "C:\\Users\\adity\\Desktop\\Thesis\\files\\selected_solutions\\1289.java"
studentSampleSolution = open(studentFilepath).read()
studentErrorCount, studentScore, studentErrorDict = check_java_syntax(studentFilepath)

Compilation Errors:
- Line 236: error: class CBT_PART_1_QP is public, should be declared in a file named CBT_PART_1_QP.java
- Line 140: error: no suitable method found for toString(String)

Compiler reported errors: 2
Number of lines with errors: 2
Score: 3.5/5

Error Dictionary:
{
  "236": [
    "error: class CBT_PART_1_QP is public, should be declared in a file named CBT_PART_1_QP.java"
  ],
  "140": [
    "error: no suitable method found for toString(String)"
  ]
}


In [8]:
studentErrorCount, studentScore, studentErrorDict

(2,
 3.5,
 {236: ['error: class CBT_PART_1_QP is public, should be declared in a file named CBT_PART_1_QP.java'],
  140: ['error: no suitable method found for toString(String)']})

In [9]:
# stageOnePrompt = replaceKeys(prompts[0]["prompt"], ["PROBLEM", "CODE"], [problem, code])
#         stageOneResponse = self.llm.invoke(stageOnePrompt).content

finalPrompt = feedbackPromptAIO.format(question, rubric, studentSampleSolution, studentErrorDict)

In [10]:
from langchain_openai import ChatOpenAI

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = llm.bind(response_format={"type": "json_object"})
response = json.loads(llm.invoke(finalPrompt).content)

KeyboardInterrupt: 

In [26]:
response

{'RunsComparator: compare Method': 2,
 'CricketDataHandler: readPlayersFromFile Method': {'Step 1: Create an empty list to store player details': 1,
  'Step 2: Open the specified file for reading data': 1,
  'Step 3: Ignore the first line since it contains the column names': 1,
  'Step 4: Read each line one by one until reaching the end of the file': 1,
  'Step 5: Split the line into different pieces of information': 1,
  'Step 6: Create a new player using this information': 1,
  'Step 7: Add the new player to the list': 1,
  'Step 8: Close the file after reading all data': 1,
  'Step 9: Return the complete list of players': 0},
 'CricketDataHandler: writePlayersToFile Method': {'Step 1: Prepare to write data into the specified file': 0,
  'Step 2: Write the column names as the first line of the file': 0,
  'Step 3: For each player in the list, convert their details to the desired format': 0,
  "Step 4: Write each player's information to the file": 0},
 'CricketDataHandler: updatePlaye

In [28]:
finalMarks = 0
def getMarks(evaluation: dict):
    global finalMarks
    for key in evaluation:
        if type(evaluation[key]) == dict:
            getMarks(evaluation[key])
        elif type(evaluation[key]) == int:
            finalMarks += evaluation[key]

getMarks(response)

In [29]:
finalMarks

29

In [36]:
def syntacticalEvalution(errorDict: dict):
    marks = 5
    penalty = 1
    errorCount = len(errorDict.keys())
    return max(marks - (penalty * errorCount), 0)

syntaxMarks = syntacticalEvalution(studentErrorDict)


In [37]:
totalMarks = finalMarks + syntaxMarks
totalMarks
# Max Marks = 35 + 5 = 40

32